*** p579~p590
## 05. 콘텐츠 기반 필터링 실습 - TMDB 5000 영화 데이터 세트
### 장르 속성을 이용한 영화 콘텐츠 기반 필터링
- 사용자가 특정 영화를 감상하고 그 영화를 좋아했다면 그 영화와 비슷한 특성, 속성, 구성 요소 등을 가진 다른 영화를 추천하는 것
- 영화(또는 상품/서비스)간의 유사성을 판단하는 기준이 영화를 구성하는 다양한 콘텐츠 (장르, 감독, 배우, 평점, 키워드, 영화설명)를 기반으로 하는 방식
- 영화 장르 속성을 기반으로 한 콘텐츠 기반 필터링 추천 시스템 구현
 - 장르 칼럼 값의 유사도를 비교한 뒤 그중 높은 평점을 가지는 영화를 추천하는 방식

### 데이터 로딩 및 가공

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

movies = pd.read_csv('/Users/jaewon/Downloads/ESAA/tmdb_5000_movies.csv')
print(movies.shape)
movies.head(1)

(4803, 20)


,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


In [6]:
movies_df = movies[['id', 'title', 'genres', 'vote_average', 'vote_count', 'popularity',
                   'keywords', 'overview']]

pd.set_option('max_colwidth', 100)
movies_df[['genres', 'keywords']][:1]

,genres,keywords
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"": 2964, ""name"": ""future""}, {""id"": 3386, ""name"": ""sp..."


In [7]:
from ast import literal_eval

# 내부에 딕셔너리가 있는 리스트 형태로 변경
movies_df['genres'] = movies_df['genres'].apply(literal_eval)
movies_df['keywords'] = movies_df['keywords'].apply(literal_eval)

# 개별 장르를 리스트 객체로 추출
movies_df['genres'] = movies_df['genres'].apply(lambda x : [ y['name'] for y in x] )
movies_df['keywords'] = movies_df['keywords'].apply(lambda x : [ y['name'] for y in x] )

movies_df[['genres', 'keywords']][:1]

,genres,keywords
0,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colony, society, space travel, futuristic, romance, spa..."


### 장르 콘텐츠 유사도 측정
- 문자열로 변환된 genres 칼럼을 count 기반으로 피처 벡터화 변환
- genres 문자열을 피처 벡터화 행렬로 변환한 데이터 세트를 코사인 유사도를 통해 비교
- 데이터 세트의 레코드별로 타 레코드와 장르에서 코사인 유사도 값을 가지는 객체를 생성
- 장르 유사도가 높은 영화 중에 평점이 높은 순으로 영화를 추천

In [9]:
from sklearn.feature_extraction.text import CountVectorizer

# CountVectorizer를 적용하기 위해 공백문자로 word단위가 구분되는 문자열로 변환
movies_df['genres_literal'] = movies_df['genres'].apply(lambda x : (' ').join(x))
count_vect = CountVectorizer(min_df=0, ngram_range=(1,2))
genre_mat = count_vect.fit_transform(movies_df['genres_literal'])
print(genre_mat.shape)

(4803, 276)


- 4803개의 레코드와 276개의 개별 단어 피처로 구성된 피처 벡터 행렬
- movies_df의 개별 레코드에 대해서 가장 장르 유사도가 높은 순으로 다른 레코드를 추출하기 위해 genre_sim 행렬 이용

In [10]:
from sklearn.metrics.pairwise import cosine_similarity

genre_sim = cosine_similarity(genre_mat, genre_mat)
print(genre_sim.shape)
print(genre_sim[:1])

(4803, 4803)
[[1.         0.59628479 0.4472136  ... 0.         0.         0.        ]]


In [12]:
# 비교 대상이 되는 행의 유사도 값이 높은 순으로 정렬된 행렬의 위치를 나타내는 인덱스 값 추출
# 각 레코드의 장르 코사인 유사도가 가장 높은 순으로 정렬된 타레코드의 위치 인덱스 값을 가진 객체
genre_sim_sorted_ind = genre_sim.argsort()[:,::-1]
print(genre_sim_sorted_ind[:1])

[[   0 3494  813 ... 3038 3037 2401]]


### 장르 콘텐츠 필터링을 이용한 영화 추천

In [15]:
# 장르 유사도에 따라 영화를 추천하는 함수 생성
def find_sim_movie(df, sorted_ind, title_name, top_n=10):
    # 인자로 입력된 movies_df DataFrame에서 'title' 칼럼이 입력된 title_name 값인 DataFrame 추출
    title_movie = df[df['title']==title_name]
    
    # title_name을 가진 DataFrame의 index 객체를 ndarray로 반환하고
    # sorted_ind 인자로 입력된 genre_sim_sorted_ind 객체에서 유사도 순으로 top_n개의 index 추출
    title_index = title_movie.index.values
    similar_indexes = sorted_ind[title_index, :(top_n)]
    
    # 추출된 top_n index 출력, top_n index는 2차원 데이터
    # DataFrame에서 index로 사용하기 위해서 1차원 array로 변경
    print(similar_indexes)
    similar_indexes = similar_indexes.reshape(-1)
    
    return df.iloc[similar_indexes]

similar_movies = find_sim_movie(movies_df, genre_sim_sorted_ind, 'The Godfather', 10)
similar_movies[['title','vote_average']]

[[2731 1243 3636 1946 2640 4065 1847 4217  883 3866]]


,title,vote_average
2731,The Godfather: Part II,8.3
1243,Mean Streets,7.2
3636,Light Sleeper,5.7
1946,The Bad Lieutenant: Port of Call - New Orleans,6.0
2640,Things to Do in Denver When You're Dead,6.7
4065,Mi America,0.0
1847,GoodFellas,8.2
4217,Kids,6.8
883,Catch Me If You Can,7.7
3866,City of God,8.1


In [16]:
movies_df[['title','vote_average','vote_count']].sort_values('vote_average', ascending=False)[:10]

,title,vote_average,vote_count
3519,Stiff Upper Lips,10.0,1
4247,Me You and Five Bucks,10.0,2
4045,"Dancer, Texas Pop. 81",10.0,1
4662,Little Big Top,10.0,1
3992,Sardaarji,9.5,2
2386,One Man's Hero,9.3,2
2970,There Goes My Baby,8.5,2
1881,The Shawshank Redemption,8.5,8205
2796,The Prisoner of Zenda,8.4,11
3337,The Godfather,8.4,5893


- 평가 횟수가 매우 작아 평점 데이터를 왜곡하는 경우 발생
- 평가 횟수에 대한 가중치가 부여된 평점 방식 사용
 - 가중 평점(weighted rating) = $(v/(v+m))*R+(m/(v+m))*C$
 - $v$ : 개별 영화에 평점을 투표한 횟수
 - $m$ : 평점을 부여하기 위한 최소 투표 횟수 - 전체 투표 횟수에서 상위 60%로 지정
 - $R$ : 개별 영화에 대한 평균 평점
 - $C$ : 전체 영화에 대한 평균 평점

In [17]:
C = movies_df['vote_average'].mean()
m = movies_df['vote_count'].quantile(0.6)
print('C:', round(C,3), 'm:', round(m,3))

C: 6.092 m: 370.2


In [19]:
percentile = 0.6
m = movies['vote_count'].quantile(percentile)
C = movies['vote_average'].mean()

def weighted_vote_average(record):
    v = record['vote_count']
    R = record['vote_average']
    
    return ((v/(v+m))*R) + ((m/(m+v))*C)

movies_df['weighted_vote']=movies_df.apply(weighted_vote_average, axis=1)

# 새롭게 부여된 가중평점이 높은 순으로 상위 10개의 영화 추출
movies_df[['title','vote_average','weighted_vote','vote_count']].sort_values('weighted_vote', ascending=False)[:10]

,title,vote_average,weighted_vote,vote_count
1881,The Shawshank Redemption,8.5,8.396052,8205
3337,The Godfather,8.4,8.263591,5893
662,Fight Club,8.3,8.216455,9413
3232,Pulp Fiction,8.3,8.207102,8428
65,The Dark Knight,8.2,8.136930,12002
1818,Schindler's List,8.3,8.126069,4329
3865,Whiplash,8.3,8.123248,4254
809,Forrest Gump,8.2,8.105954,7927
2294,Spirited Away,8.3,8.105867,3840
2731,The Godfather: Part II,8.3,8.079586,3338


In [20]:
def find_sim_move(df, sorted_ind, title_name, top_n=10):
    title_movie = df[df['title']==title_name]
    title_index = title_movie.index.values
    
    # top_n의 2배에 해당하는 장르 유사성이 높은 인덱스 추출
    similar_indexes = sorted_ind[title_index, :(top_n*2)]
    similar_indexes = similar_indexes.reshape(-1)
    
    # 기준 영화 인덱스는 제외
    similar_indexes = similar_indexes[similar_indexes != title_index]
    
    # top_n의 2배에 해당하는 후보군에서 weighted_vote가 높은 순으로 top_n만큼 추출
    return df.iloc[similar_indexes].sort_values('weighted_vote', ascending=False)[:top_n]

similar_movies = find_sim_movie(movies_df, genre_sim_sorted_ind, 'The Godfather', 10)
similar_movies[['title', 'vote_average', 'weighted_vote']]

[[2731 1243 3636 1946 2640 4065 1847 4217  883 3866]]


,title,vote_average,weighted_vote
2731,The Godfather: Part II,8.3,8.079586
1243,Mean Streets,7.2,6.626569
3636,Light Sleeper,5.7,6.076900
1946,The Bad Lieutenant: Port of Call - New Orleans,6.0,6.049012
2640,Things to Do in Denver When You're Dead,6.7,6.205672
4065,Mi America,0.0,6.092172
1847,GoodFellas,8.2,7.976937
4217,Kids,6.8,6.396368
883,Catch Me If You Can,7.7,7.557097
3866,City of God,8.1,7.759693


- 본 콘텐츠 필터링 예제를 좀 더 다양한 콘텐츠 기반으로 확장 가능함